In [1]:
import os
current_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

import sys
sys.path.append(parent_dir)

In [2]:
import idx2numpy
import numpy as np

#from torchvision import transforms
from torchvision.transforms import v2 # for torchvision > 0.15
import torch.nn as nn
import torcheval.metrics as metrics

%matplotlib inline 
import matplotlib.pyplot as plt

from utils.dataloaders import *
from utils.preprocessing import *
from utils.training_utils import *
from utils.metrics import *
from utils.losses import *

from autoencoder.models import *
from datetime import date

# Load datasets

In [ ]:
data_dir = "../datasets/MNIST"

In [ ]:
train_images = idx2numpy.convert_from_file(os.path.join(data_dir, "train", "train-images-idx3-ubyte"))
train_labels = idx2numpy.convert_from_file(os.path.join(data_dir, "train", "train-labels-idx1-ubyte"))
test_images = idx2numpy.convert_from_file(os.path.join(data_dir, "test", "t10k-images-idx3-ubyte"))
test_labels = idx2numpy.convert_from_file(os.path.join(data_dir, "test", "t10k-labels-idx1-ubyte"))

In [ ]:
# Data stats
print("[INFO]: Training set shape: {}".format(train_images.shape))
print("[INFO]: Test set shape: {}".format(test_images.shape))
print("[INFO]: Training label shape: {}".format(train_labels.shape))
print("[INFO]: Test label shape: {}".format(test_labels.shape))

[INFO]: Training set shape: (60000, 28, 28)
[INFO]: Test set shape: (10000, 28, 28)
[INFO]: Training label shape: (60000,)
[INFO]: Test label shape: (10000,)


# Preprocessing Data

In [ ]:
# Image scaling
train_images = train_images/255
test_images = test_images/255

# Calculate mean and std
mean = np.mean(train_images)
std = np.std(train_images)

In [ ]:
train_transform = v2.Compose([
    v2.ToTensor(), # deprecated # Will transpose the image, since torch expected the input image to be of shape [H, W, C] -> [C, H, W]
    v2.Normalize(mean=[mean], std=[std]),
    v2.ToDtype(torch.float32, scale=False),
])

c:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [ ]:
train_dataset = MnistDataset(images=train_images, labels=train_labels, transforms=train_transform)
test_dataset = MnistDataset(images=test_images, labels=test_labels, transforms=train_transform)

# Define training configurations

In [ ]:
# Model configuration
in_channels = 1
enc_hidden_channels = 16
emb_channels = 4
dec_hidden_channels = 16

# Training configuration
epochs = 25
batchsize = 128
learning_rate = 0.001
num_workers = 4
device = "cuda"
use_wandb = True
use_tensorboard = True

# Checkpoint dir
save_dir = ".\checkpoints"
exp_name = "conv_autoencoder_{}".format(str(date.today()))
save_dir = os.path.join(save_dir, exp_name)
if not os.path.exists(save_dir):
    os.system("mkdir {}".format(save_dir))

In [ ]:
train_loader = get_loader(train_dataset, batchsize, num_workers, shuffle=True, drop_last=False)
test_loader = get_loader(test_dataset, batchsize, num_workers, shuffle=False, drop_last=False)

In [ ]:
model = Conv_AutoEncoder(in_channels, enc_hidden_channels, emb_channels, dec_hidden_channels)

In [ ]:
loss_func = nn.MSELoss()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
metrics = [
    # The first thre metrics only work on images
    # metrics.FrechetInceptionDistance(device=device), # Only work on three-channel images
    # metrics.StructuralSimilarity(device=device),
    metrics.PeakSignalNoiseRatio(device=device),
    # metrics.MeanSquaredError(device=device)
]
metric_weights = [1.0]

In [ ]:
if use_wandb:
    # initialize wandb for usage
    wandb.login()
    wandb.init(
        # Set the project where this run will be logged
        project="conv-mnist-autoencoder", 
        # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
        name=exp_name, 
        # Track hyperparameters and run metadata
        config={
            "learning_rate": learning_rate,
            "architecture": str(model.__class__.__name__),
            "dataset": "MNIST",
            "epochs": epochs,
        })

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: quyenobest2000. Use `wandb login --relogin` to force relogin


In [ ]:
train_epochs(
    model=model,
    train_loader=train_loader,
    val_loader=test_loader,
    optimizer=optimizer,
    loss_func=loss_func,
    metrics=metrics,
    metric_weights=metric_weights,
    device=device,
    num_epochs=epochs,
    log_rate=5,
    save_rate=10,
    save_dir=save_dir,
    use_tensorboard=use_tensorboard,
    use_wandb=use_wandb,
    interval=0
)

torch.Size([128, 1, 28, 28])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
         MaxPool2d-2           [-1, 16, 14, 14]               0
            Conv2d-3            [-1, 4, 14, 14]             580
         MaxPool2d-4              [-1, 4, 7, 7]               0
      Conv_Encoder-5              [-1, 4, 7, 7]               0
   ConvTranspose2d-6           [-1, 16, 14, 14]             272
   ConvTranspose2d-7            [-1, 1, 28, 28]              65
      Conv_Decoder-8            [-1, 1, 28, 28]               0
Total params: 1,077
Trainable params: 1,077
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.16
Params size (MB): 0.00
Estimated Total Size (MB): 0.17
----------------------------------------------------------------
[INFO]: Epoch: 1/

0it [00:00, ?it/s]


ValueError: The dimension `input` and `target` should be 1D or 2D, got shapes torch.Size([128, 1, 28, 28]) and torch.Size([128, 1, 28, 28]).